In [1]:
import os
os.environ['OPENBLAS_NUM_THREADS'] = '1'
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
from multiprocessing import Pool, shared_memory

### make sure the proc_num is around 2/3 of the number of your machine's cores

In [6]:
proc_num = 55

In [3]:
def sample_matrix(n, p_diag, p_off_diag):
    """Sample a random matrix
       n:          the size of the matrix (n,n).
       p_diag:     The function that samples n elements for diagonal terms
                   it must have a signiture like p_diag(n).
       p_off_diag: The function that samples (n,n) elements for off-diagonal terms
                   it must have a signiture like p_off_diag(n).
    """
        
    # Create an n by n matrices, sampled from 'p_off_diag' distribution
    m = p_off_diag(n)
    # Fill the diagonal terms by sampling from 'p_diag' distribution
    off_diag_indices = np.arange(0,n).astype(np.int32)
    m[off_diag_indices, off_diag_indices] = p_diag(n)
    
    return m

def matrix_intervals(m, axis=1):
    """Turn a matrix to row(column)-wise intervals
    
       args:
           m: (n,n) numpy array.
           axis:  use one for row-wise and zero for
                  column-wise intervals.
    """
    # Get the diagonal terms
    ds = np.diag(m)
    # Sum of absolute value of off-diagonal elements
    rs = np.sum(np.abs(m - np.diag(ds)), axis = axis)
    # create intrvals around the diagonal terms
    return np.array([ (d-r, d+r) for d, r in zip(ds, rs) ])

def alg1(m, axis=1):
    """Specifies the stabaility property of the matrix.
    
       args:
           m: (n,n) numpy array.
           axis:  use one for row-wise and zero for
                  column-wise intervals.
       return:
               0 - super-stable
               1 - inconclusive
               2 - unstable
    
    """
    n = m.shape[0]
    intervals = matrix_intervals(m,axis)
    us = np.array([u for _,u in intervals])
    ls = np.array([l for l,_ in intervals])
    u_max_index = np.argmax(us)
    u_max = us[u_max_index]    
    if u_max < 0:# Super-stable
        return 0
    
    l_i = ls[u_max_index] 
    if l_i < 0:# Inconclusive
        return 1
    indices = [i for i in range(0, n) if i != u_max_index]
    for j in indices:
        l_j, u_j = ls[j], us[j]
        if l_i < u_j:
            if l_j < l_i:
                l_i = l_j
            if l_i < 0:# Inconclusive
                return 1
    return 2# unstable            

def alg2(m, axis=1):
    """Specifies if the stabaility property can be tightened.
    
       args:
           m: (n,n) numpy array.
           axis:  use one for row-wise and zero for
                  column-wise intervals.
       return:
               0 - super-stable
               1 - inconclusive
               2 - unstable
    
    """
    n = m.shape[0]
    # Get the diagonal terms
    ds = np.diag(m)
    # Sum of absolute value of off-diagonal elements
    rs = np.sum(np.abs(m - np.diag(ds)), axis = axis)
    a_max_index = np.argmax(ds)
    a_ii = ds[a_max_index]
    r_i = rs[a_max_index]
    indices = [i for i in range(0, n) if i != a_max_index]
    if a_ii > 0:        
        for j in indices:
            r_j = rs[j]
            a_jj = m[j,j]
            if axis == 0:
                a_ji = m[j, a_max_index]
            else:
                a_ji = m[a_max_index, j]
            c_0 = r_i
            c_1 = a_jj-a_ii+r_j-np.abs(a_ji)
            c_2 = np.abs(a_ji)                        
            d_max = np.real(np.max(np.roots([c_2, c_1, c_0])))
            if d_max <= r_i/a_ii:
                return 1 # Inconclusive
            if c_1 >= 0:                
                return 1 # Inconclusive
            if c_1*c_1 <= 4*np.abs(a_ji*r_i):                
                return 1 # Inconclusive
        return 2 # Unstable
    else:
        #r_j = rs[j]
        #a_jj = m[j,j]
        #a_ji = m[j, a_max_index]
        #if r_i/np.abs(a_ii) >= np.min([ (np.abs(a_jj)-r_j)/np.abs(a_ji)  for j in indices]):
        # select r_i, a_jj and a_ji
        if axis == 0:
            parts = [(rs[j], m[j,j], m[j, a_max_index]) for j in indices]
        else:
            parts = [(rs[j], m[j,j], m[a_max_index, j]) for j in indices]
        values = [(np.abs(a_jj)-r_j)/np.abs(a_ji)  for (r_j, a_jj, a_ji) in parts]
        if r_i/np.abs(a_ii) >= np.min(values):
            return 1 # Inconclusive
        return 0 # Super-stable

def alg(m):
    ret = alg1(m, axis = 0)
    if ret != 1:
        return ret
    ret = alg1(m, axis = 1)
    if ret != 1:
        return ret
    ret = alg2(m, axis = 0)
    if ret != 1:
        return ret
    return alg2(m, axis = 1)

In [4]:
def p_diag_uniform(low):
    def uniform(n):
        return np.random.uniform(low, 0, n)    
    return uniform

def p_off_diag_exp(lamb):
    def exp(n):        
        p = np.random.uniform(0, 1)        
        return (
            np.random.exponential(1/lamb, (n,n))*
            np.where(np.random.binomial(1, p, (n, n)) == 0, -1, 1)            
        )
    return exp

def p_off_diag_normal(mean, std):
    def normal(n):        
        return np.random.normal(mean, std, (n,n))
    return normal

def is_unstable(m):
    """Find the linear stability of a matrix"""
    return np.any(np.real(sp.linalg.eigvals(m)) > 0)

# Sampling off-diagonal terms from a exponential distribution

In [24]:
ns = np.array([4, 8, 16, 32, 64, 128, 256, 512, 1024])
c = 1
lambs = np.array([ c*10, c*50,  1000*c])
trials = 1000
repeats = 5
def proc_stability1(args):
    rng = np.random.default_rng()
    np.random.seed(rng.integers(low=0, high=np.iinfo(np.int32).max))
    
    n, lamb, c, trials, repeat_id = args
    super_stable_count = 0
    stable_count = 0
    for i in range(trials):
        # All the of-diagonal elements are non-zero
        # Also, the off-diagonal rate is scaled by sqrt(n)
        m = sample_matrix(n, 
                          p_diag_uniform(-c), 
                          p_off_diag_exp(lamb))
        region1 = alg1(m, axis=0)
        #region2 = alg1(m, axis=1)
        if region1 == 0:# or region2 == 0:
            super_stable_count += 1
        else:
            if not is_unstable(m):
                stable_count += 1
        
    
    
    return (repeat_id, super_stable_count/trials, stable_count/trials)

params1 = [ (n, lamb, c, trials, repeat_id) 
           for n in ns for lamb in lambs for repeat_id in range(repeats)]
with Pool(proc_num) as pool:
    res1 = pool.map(proc_stability1, params1)
    
np.save("P_I_P_S_expo_no_scaling.npy",res1)

# Sampling off-diagonal terms from a folded-normal distribution

In [25]:
ns = np.array([4, 8, 16, 32, 64, 128, 256, 512, 1024])
c = 1
stds = np.array([ 0.01, 0.1, 1])
trials = 1000
repeats = 5

def proc_stability2(args):
    rng = np.random.default_rng()
    np.random.seed(rng.integers(low=0, high=np.iinfo(np.int32).max))
    
    n, std, c, trials, repeat_id = args
    super_stable_count = 0
    stable_count = 0
    for i in range(trials):
        # All the of-diagonal elements are non-zero
        # Also, the off-diagonal rate is scaled by sqrt(n)
        m = sample_matrix(n, 
                          p_diag_uniform(-c), 
                          p_off_diag_normal(0, std))
        region1 = alg1(m, axis=0)
        #region2 = alg1(m, axis=1)
        if region1 == 0:# or region2 == 0:
            super_stable_count += 1
        else:
            if not is_unstable(m):
                stable_count += 1
        
    
    
    return (repeat_id, super_stable_count/trials, stable_count/trials)

params2 = [ (n, std, c, trials, repeat_id) 
           for n in ns for std in stds for repeat_id in range(repeats)]
with Pool(proc_num) as pool:
    res2 = pool.map(proc_stability2, params2)
    
np.save("P_I_P_S_normal_no_scaling.npy",res2)

In [26]:
ns = np.array([4, 8, 16, 32, 64, 128, 256, 512, 1024])
c = 100
stds = np.array([ 0.01, 0.1, 1])
trials = 1000
repeats = 5

def proc_stability3(args):
    rng = np.random.default_rng()
    np.random.seed(rng.integers(low=0, high=np.iinfo(np.int32).max))
    
    n, std, c, trials, repeat_id = args
    super_stable_count = 0
    stable_count = 0
    for i in range(trials):
        # All the of-diagonal elements are non-zero
        # Also, the off-diagonal rate is scaled by sqrt(n)
        m = sample_matrix(n, 
                          p_diag_uniform(-c), 
                          p_off_diag_normal(0, std))
        region1 = alg1(m, axis=0)
        #region2 = alg1(m, axis=1)
        if region1 == 0:# or region2 == 0:
            super_stable_count += 1
        else:
            if not is_unstable(m):
                stable_count += 1
        
    
    
    return (repeat_id, super_stable_count/trials, stable_count/trials)

params3 = [ (n, std, c, trials, repeat_id) 
           for n in ns for std in stds for repeat_id in range(repeats)]
with Pool(proc_num) as pool:
    res3 = pool.map(proc_stability3, params3)
    
np.save("P_I_P_S_normal_no_scaling_c_100.npy",res3)